## Step-2
Fitness Score - measure the dissimilarity of the objective function and random question paper

In [9]:
def fittness_score(objective_function,Question_paper):
    Fittness_score =[]
    Question_label=[]
    for k in range(total_number_of_question_paper):
    
        label=list(Question_paper[k].Label)
    
        Question_label.append(label)
    
    for j in range(len(Question_label)):
    
        label,count=np.unique(Question_label[j],return_counts=True)
        label_in_percentage=[]
        for i in range(len(count)):
            label_in_percentage.append(int((count[i]/Number_of_question)*100))

        sample_question_bloom_taxonomy_label= {label[i]: label_in_percentage[i] for i in range(len(label))} 
        bloom_taxonomy_label=list(objective_function.keys())
        fittness=0
        for i in range(len(bloom_taxonomy_label)):
     
             if bloom_taxonomy_label[i] in sample_question_bloom_taxonomy_label:
            
                    error = abs(objective_function[bloom_taxonomy_label[i]] - sample_question_bloom_taxonomy_label[bloom_taxonomy_label[i]])
                    fittness=fittness+error
            
             else:
            
                    fittness=fittness+objective_function[bloom_taxonomy_label[i]]
        Fittness_score.append(fittness)    
    return Fittness_score
        

# Step-3
Select the parent which have low fitness score

In [10]:
num_parents=4

def select_mating_pool(pop, fitness, num_parents):

    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.

    parents=[[] for j in range(num_parents)]

    for parent_num in range(num_parents):

        max_fitness_idx = np.where(fitness == np.min(fitness))

        max_fitness_idx = max_fitness_idx[0][0]

        parents[parent_num] = (pop[max_fitness_idx])

        fitness[max_fitness_idx] = 99999999999

    return parents

#parents=select_mating_pool(Question_paper,Fittness_score,num_parents)


# Step-4
Crossover- select 2 parent 

In [11]:
#offspring_size=total_number_of_question_paper - num_parents

def crossover(parents,offspring_size):
    offspring=[[] for j in range(offspring_size)]
    #offspring = np.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = np.uint8(Number_of_question/2)

    for k in range(len(offspring)):
        # Index of the first parent to mate.
        parent1_idx = k%len(parents)
        
        # Index of the second parent to mate.
        parent2_idx = (k+1)%len(parents)
        
        # The new offspring will have its first half of its genes taken from the first parent.
        upper_ques= (parents[parent1_idx][0:crossover_point])

        # The new offspring will have its second half of its genes taken from the second parent.
        lower_ques= (parents[parent2_idx][crossover_point:])
        offspring[k]=pd.concat([upper_ques,lower_ques],axis=0,sort=False)

   
    return offspring  


# step-5
Mutation process - replace one question of each paper randomly from the question bank

In [12]:
def mutation(offspring_crossover,sample_data):

    # Mutation changes a single gene in each offspring randomly.

    for idx in range(len(offspring_crossover)):

        # The random value to be added to the gene.

        #random_value = numpy.random.uniform(-1.0, 1.0, 1)
        random_value=np.array(((sample_data.sample())))
        
        arr=np.array(offspring_crossover[idx])
        arr[4] = random_value
        offspring_crossover[idx]=pd.DataFrame(arr)
        offspring_crossover[idx].rename(columns = {0:'Question', 1:'Label'}, inplace = True) 
   
        

    return offspring_crossover
#offspring_crossover=mutation(offspring,sample_data)

# sample code
Taking 5 iteration using for loop

In [13]:
import numpy as np
import pandas as pd
import math
import random

data=pd.read_csv('C:/Users/SRIKANT NAYAK/Documents/code/bloom_texonomy/data1.csv',encoding= 'unicode_escape')
df=pd.DataFrame(data)

shuffled_data = df.sample(frac=1)

sample_data = shuffled_data[:100]

Number_of_question = 10

objective_function = {'Analysis':10 ,'Application':10 , 'Comprehension':15 ,'Evaluation ' :30,'Knowledge':15 ,'Synthesis':20 }

bloom_taxonomy_label=list(objective_function.keys())

value_in_percentage=list(objective_function.values())

total_number_of_question_paper = int(sample_data.shape[0]/Number_of_question)

#initialization ....

Question_paper=[]


for i in range(total_number_of_question_paper):
    
    Question_paper.append(sample_data[i*total_number_of_question_paper:(i+1)*total_number_of_question_paper])
    

    
    
fittness_scorre=fittness_score(objective_function,Question_paper) 

num_generations=5
num_parents=4
offspring_size=total_number_of_question_paper - num_parents



for generation in range(num_generations):
    
    print("Generation : ", generation)
    
    # Measing the fitness of each chromosome in the population.
    fitness = fittness_score(objective_function,Question_paper)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(Question_paper, fitness, num_parents)
    

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,offspring_size)

    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover,sample_data)
  
    
    

    # Creating the new population based on the parents and offspring.
    Question_paper[0:len(parents)] = parents
    Question_paper[len(parents):] = offspring_mutation
    #Question_paper=pd.concat([parents,offspring_mutation],axis=0)
    

    # The best result in the current iteration.
    print("Best result : ", np.min(fitness))
    
    

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = fittness_score(objective_function,Question_paper)
print(fitness)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.min(fitness))
best_match_idx=np.array(best_match_idx).astype(int)
best_match_idx=(best_match_idx[0])

print("Best solution : ", Question_paper[best_match_idx[0]])
#print("Best solution : ", Question_label[best_match_idx[0]])
print("Best solution fitness : ", fitness[best_match_idx[0]])

Generation :  0
Best result :  50
Generation :  1
Best result :  30
Generation :  2
Best result :  30
Generation :  3
Best result :  30
Generation :  4
Best result :  30
[10, 20, 30, 30, 30, 50, 40, 70, 40, 50]
Best solution :                                                Question          Label
117           Can you defend your position about ... ?    Evaluation 
31   Explain why the story has the title that it does.  Comprehension
139  Your advice has been sought to settle the foll...    Evaluation 
132             What judgments can you make about...?     Evaluation 
25      What is the definition of the following terms      Knowledge
56       Write a textbook about this topic for others.    Application
94              Design a building to house your study.      Synthesis
104  Write a journal from the point of view of moun...      Synthesis
43   Identify three mistakes from the passage and c...  Comprehension
81      What has the author used to create this effect       Analysis
Be

In [19]:
fittness_scorre

[40, 50, 70, 30, 60, 10, 30, 60, 50, 70]

# Genratic algorithm


In [14]:
def Question_paper_generation(data,objective_function,Number_of_question,num_parents,threshold,max_generation):

    df=pd.DataFrame(data)

    shuffled_data=df.sample(frac=1)

    sample_data=shuffled_data[:100]


    total_number_of_question_paper = int(sample_data.shape[0]/Number_of_question)

    #initialization ....

    Question_paper=[]


    for i in range(total_number_of_question_paper):

        Question_paper.append(sample_data[i*total_number_of_question_paper:(i+1)*total_number_of_question_paper])




    #fittness_scorre=fittness_score(objective_function,Question_paper) 


    offspring_size=total_number_of_question_paper - num_parents
    generation=0
    while(1):

    #for generation in range(num_generations):

        print("Generation : ", generation)

        # Measing the fitness of each chromosome in the population.
        fitness = fittness_score(objective_function,Question_paper)

        # Selecting the best parents in the population for mating.
        parents = select_mating_pool(Question_paper, fitness, num_parents)


        # Generating next generation using crossover.
        offspring_crossover = crossover(parents,offspring_size)

        # Adding some variations to the offsrping using mutation.
        offspring_mutation = mutation(offspring_crossover,sample_data)



        new_question=[[] for i in range(total_number_of_question_paper)]
        # Creating the new population based on the parents and offspring.
        new_question[0:len(parents)] = parents
        new_question[len(parents):] = offspring_mutation
        #Question_paper=pd.concat([parents,offspring_mutation],axis=0)

        best_result=np.min(fitness)

        generation=generation+1

        if (best_result<=threshold):
            break
        if (generation>=max_generation):
            break

        Question_paper = new_question
        # The best result in the current iteration.
        print("Best result : ", np.min(fitness))



    # Getting the best solution after iterating finishing all generations.
    #At first, the fitness is calculated for each solution in the final generation.
    fitness = fittness_score(objective_function,Question_paper)
    print(fitness)
    # Then return the index of that solution corresponding to the best fitness.
    best_match_idx = np.where(fitness == np.min(fitness))
    best_match_idx=np.array(best_match_idx).astype(int)
    best_match_idx=(best_match_idx[0])
    print(best_match_idx)
    print("Best solution : ", Question_paper[best_match_idx[0]])
    #print("Best solution : ", Question_label[best_match_idx[0]])
    print("Best solution fitness : ", fitness[best_match_idx[0]])
    
    return pd.DataFrame(Question_paper[best_match_idx[0]])



# New Question paper generation according to the bloom taxonomy

In [15]:
import numpy as np
import pandas as pd
import math
import random

data=pd.read_csv('C:/Users/SRIKANT NAYAK/Documents/code/bloom_texonomy/data1.csv',encoding= 'unicode_escape')



objective_function = {'Analysis':10 ,'Application':10 , 'Comprehension':10 ,'Evaluation ' :30,'Knowledge':20 ,'Synthesis':20 }
Number_of_question = 10

num_parents=4 # number of selelected question paper which have lowest fitness value

threshold=10 # condition for the error should be minimum 

max_generation=100 #maximum number of iteration 

ques_paper=Question_paper_generation(data,objective_function,Number_of_question,num_parents,threshold,max_generation)






Generation :  0
Best result :  60
Generation :  1
Best result :  40
Generation :  2
Best result :  20
Generation :  3
Best result :  20
Generation :  4
Best result :  20
Generation :  5
[0, 0, 0, 0, 20, 20, 20, 20, 0, 0]
[0 1 2 3 8 9]
Best solution :                                              Question          Label
0  List two reference parameters in the setHour f...      Knowledge
1        Write about your feelings in relation to...      Synthesis
2  Analyze a distinction. State the point of view...       Analysis
3  Prepare a list of criteria to judge ... Evalua...    Evaluation 
4  Rewrite the story from an animals point of view.      Synthesis
5  Demonstrate and illustrate measures of central...    Application
6  How would you rank the items given the criteria ?    Evaluation 
7                             Define Method in JAVA.      Knowledge
8     Describe how interest rates affect the economy  Comprehension
9  What would you cite to defend the actions . . ...    Evaluation 


In [16]:
ques_paper

,Question,Label
0,List two reference parameters in the setHour f...,Knowledge
1,Write about your feelings in relation to...,Synthesis
2,Analyze a distinction. State the point of view...,Analysis
3,Prepare a list of criteria to judge ... Evalua...,Evaluation
4,Rewrite the story from an animals point of view.,Synthesis
5,Demonstrate and illustrate measures of central...,Application
6,How would you rank the items given the criteria ?,Evaluation
7,Define Method in JAVA.,Knowledge
8,Describe how interest rates affect the economy,Comprehension
9,What would you cite to defend the actions . . ...,Evaluation
